In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Models and selection methods
from sklearn.base import clone
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
# Binary classifier metrics
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
# Linear regression metrics
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
#Pré-Processamento de dados
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from sklearn.compose import ColumnTransformer

In [3]:
# Estatisticas para classificadores
def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    matthews = matthews_corrcoef(truth, preds)
    print("The Matthews correlation coefficient is: %7.4f" % matthews)
    print()
    print("This is the Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(truth, preds)))

# Previsao de resultados com cross validation
def CrossValidation(X_TRAIN, y_TRAIN, kf, model):
    TRUTH=None
    PREDS=None
    for train_index, test_index in kf.split(X_TRAIN):
        X_train, X_test = X_TRAIN[train_index], X_TRAIN[test_index]
        y_train, y_test = y_TRAIN[train_index], y_TRAIN[test_index]
        temp_model = clone(model)
        temp_model.fit(X_train, y_train)
        preds = temp_model.predict(X_test)
        if TRUTH is None:
            PREDS=preds
            TRUTH=y_test
        else:
            PREDS=np.hstack((PREDS, preds))
            TRUTH=np.hstack((TRUTH, y_test))
    return (TRUTH, PREDS)
    
# Model testing rapido
def naif_model_testing(X_train, X_test, y_train, y_test):
    rfr= RandomForestClassifier(n_jobs=-1)
    rfr.fit(X_train, y_train)
    dtr= DecisionTreeClassifier(max_depth=5)
    dtr.fit(X_train, y_train)
    lmr=LogisticRegression(n_jobs=-1)
    lmr.fit(X_train, y_train)
    rf_preds=rfr.predict(X_test)
    dt_preds=dtr.predict(X_test)
    lr_preds=lmr.predict(X_test)
    scores = [f1_score(y_test, rf_preds),f1_score(y_test, dt_preds),f1_score(y_test, lr_preds)]
    print("F1 RFs: %7.4f" % f1_score(y_test, rf_preds))
    print("F1 DTs: %7.4f" % f1_score(y_test, dt_preds))
    print("F1 LRs: %7.4f" % f1_score(y_test, lr_preds))
    print("F1 Avg:  %7.4f" % (sum(scores) / len(scores)))
    return (sum(scores) / len(scores))

#Escolhas de features metodo 
def Step_for(X_train, X_test, y_train, y_test):
    
    N,M=X_train.shape

    #Vamos usar random forests
    rfr=RandomForestClassifier(random_state=45)
    sfs = SequentialFeatureSelector(rfr, n_features_to_select=10)
    sfs.fit(X_train, y_train)

    #get the relevant columns
    features=sfs.get_support()
    Features_selected =np.arange(M)[features]
    print("The features selected are columns: ", Features_selected)

    nX_train=sfs.transform(X_train)
    nX_test=sfs.transform(X_test)

    f1_avg = naif_model_testing(nX_train, nX_test, y_train, y_test)
    return (f1_avg, nX_train, nX_test)
    
def ML_Sel(X_train, X_test, y_train, y_test, thresh):
    N,M=X_train.shape

    rfr=RandomForestClassifier(random_state=45, n_jobs=-1)
    sel = SelectFromModel(estimator=rfr, threshold=thresh)
    sel.fit(X_train, y_train)
    
    print("Default threshold: ", sel.threshold_)
    features=sel.get_support()
    Features_selected =np.arange(M)[features]
    print("The features selected are columns: ", Features_selected)
    nX_train=sel.transform(X_train)
    nX_test=sel.transform(X_test)
    f1_avg = naif_model_testing(nX_train, nX_test, y_train, y_test)
    return (f1_avg, nX_train, nX_test)

#Função de peso para o KNN
def gaussian(dsts):
    kernel_width = .5
    weights = np.exp(-(dsts**2)/kernel_width)
    return weights

## Pré-Processamento dos dados
Preparação do dataset - importação, normalização e preenchimento dos missing values

In [4]:
#Criar dataframe
bio_a = pd.read_csv('biodegradable_a.csv')
#Separação das 41 variáveis do y
X_bio_a=bio_a.drop(columns=["Biodegradable"])
y_bio_a=bio_a['Biodegradable'].apply(lambda x : 1 if x == 'RB' else 0)
#Converter para numpy array
Xc_bio= X_bio_a.to_numpy()
yc_bio= y_bio_a.to_numpy()
# Divisão do dataset em training set e independent validation set
X_bio_train, X_bio_test, y_bio_train, y_bio_test = train_test_split(Xc_bio, yc_bio, test_size=0.25, random_state=512)
# Kfold
kf = KFold(n_splits=16, shuffle=True, random_state = 274)

In [5]:
# Converter os tipos de floats para outros, para puder determinar as categoricas pelo tipo Int64
X_train_temp = pd.DataFrame(X_bio_train).convert_dtypes()
X_test_temp = pd.DataFrame(X_bio_test).convert_dtypes()

# Imputting das variaveis categoricas utilizando a moda
imputer_categoricas = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Selecionar as colunas com variaveis Int64
imputer_categoricas.fit(X_train_temp.select_dtypes("Int64"))
#Criar um dataframe com as varaiveis categoricas sem missing values
X_train_categorical = imputer_categoricas.transform(X_train_temp.select_dtypes("Int64"))
X_test_categorical = imputer_categoricas.transform(X_test_temp.select_dtypes("Int64"))

#Criar um dataframe com as variaveis continuas
X_train_continuos = X_train_temp.select_dtypes(exclude="Int64")
X_test_continuos = X_test_temp.select_dtypes(exclude="Int64")

# Juntar os dataframes das categoricas e continuas com os indices de coluna originais, dar sort das colunas por indice, e converter para float para puder fazer scaling
X_bio_train_step1 = X_train_continuos.join(pd.DataFrame(X_train_categorical, columns=X_train_temp.select_dtypes("Int64").columns)).sort_index(axis=1).astype(float)
X_bio_test_step1 = X_test_continuos.join(pd.DataFrame(X_test_categorical, columns=X_train_temp.select_dtypes("Int64").columns)).sort_index(axis=1).astype(float)

Passamos agora à normalização dos dados. Vão ser escolhidos os seguintes métodos de normalização para comparar mais tarde: MinMax Scaler, Standard Scaler e Power Transformer

In [6]:
#Passemos à normalização dos dados
scaler_p = PowerTransformer()
scaler_st = StandardScaler()
scaler_min= MinMaxScaler()

#Transformar dados
X_bio_train_p=scaler_p.fit_transform(X_bio_train_step1)
X_bio_test_p=scaler_p.fit_transform(X_bio_test_step1)

X_bio_train_st=scaler_st.fit_transform(X_bio_train_step1)
X_bio_test_st=scaler_st.fit_transform(X_bio_test_step1)

X_bio_train_min=scaler_min.fit_transform(X_bio_train_step1)
X_bio_test_min=scaler_min.fit_transform(X_bio_test_step1)

pd.DataFrame(X_bio_train_p)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,-1.857579,0.168452,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,-0.176119,-1.048452,1.235903,...,-0.110105,-0.457263,-0.427509,1.321538,-0.636159,2.246219,-0.468445,-0.949614,-0.222886,-0.214078
1,-1.854459,-1.019003,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,NaN,1.001919,-0.615911,...,-0.110105,-0.457263,-0.427509,1.553357,-2.135502,0.987608,2.123623,-2.097699,-0.222886,-0.214078
2,0.430395,1.193057,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,0.291224,-1.048452,0.109793,...,-0.110105,-0.457263,-0.427509,-0.874182,0.281114,0.540742,-0.468445,0.064084,-0.222886,-0.214078
3,0.421222,-0.036376,-0.298474,-0.087486,-0.329914,-0.193061,1.293260,1.092849,-1.048452,0.109793,...,-0.110105,-0.457263,-0.427509,-0.874182,0.456898,0.227531,-0.468445,0.252883,-0.222886,-0.214078
4,-0.751657,-1.775834,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,-0.466588,0.350802,-1.585803,...,-0.110105,-0.457263,-0.427509,0.821645,NaN,-1.140070,-0.468445,-1.096429,-0.222886,-0.214078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3418,1.664765,0.591478,3.349907,-0.087486,-0.329914,-0.193061,1.293260,NaN,-1.048452,0.711807,...,-0.110105,-0.457263,-0.427509,1.321538,2.082672,NaN,-0.468445,1.506523,-0.222886,-0.214078
3419,0.244807,-0.641362,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,-0.292831,1.001919,0.109793,...,-0.110105,-0.457263,-0.427509,1.553357,-0.191679,-0.552582,-0.468445,0.648379,-0.222886,-0.214078
3420,-0.562774,0.804821,-0.298474,-0.087486,-0.329914,-0.193061,-0.766057,NaN,0.350802,0.109793,...,-0.110105,-0.457263,-0.427509,1.321538,0.404293,2.169303,-0.468445,-0.276885,-0.222886,-0.214078
3421,0.755840,-0.060657,-0.298474,-0.087486,-0.329914,-0.193061,1.293260,0.813510,1.402128,-1.585803,...,-0.110105,-0.457263,-0.427509,-0.874182,0.397339,-1.456878,-0.468445,0.039927,-0.222886,-0.214078


Com os dados normalizados, podemos passar à imputação dos valores em falta. Escolheu-se o método de imputação utilizando o K-Nearest Neighbours em todos 

In [7]:
#Tratamento dos Missing values -> Utilizar Imputação de KNN

imputer_continuas = KNNImputer(n_neighbors=3, weights="uniform")


datasets_scaled = []

#imputer.fit(X_bio_train_p)
#X_bio_train_p=imputer.transform(X_bio_train_p)
#X_bio_test_p=imputer.transform(X_bio_test_p)
#datasets_scaled.append(("PowerTransformer",X_bio_train_p, X_bio_test_p))

imputer_continuas.fit(X_bio_train_st)
imputer_categoricas.fit(X_bio_train_st)
X_bio_train_st=imputer_continuas.transform(X_bio_train_st)
X_bio_test_st=imputer_continuas.transform(X_bio_test_st)
datasets_scaled.append(("StandardScaler",X_bio_train_st, X_bio_test_st))

#imputer.fit(X_bio_train_min)
#X_bio_train_min=imputer.transform(X_bio_train_min)
#X_bio_test_min=imputer.transform(X_bio_test_min)
#datasets_scaled.append(("MinMax",X_bio_train_min, X_bio_test_min))

Resta apenas ver quais são as variáveis mais relevantes. Para tal, vamos utilizar dois métodos diferentes e posteriormente comparar: Stepwise Feature Selection e Random Forests para a seleção de Features

In [8]:
datasets_reduced = []
for name, x_train_scaled, x_test_scaled in datasets_scaled:
    print("Scaling:", name)
    #print("Stepwise")
    #datasets_reduced.append((name,Step_for(x_train_scaled, x_test_scaled, y_bio_train, y_bio_test)))
    print("Random Forests")
    datasets_reduced.append(ML_Sel(x_train_scaled, x_test_scaled, y_bio_train, y_bio_test, 0.035))

#O melhor deu o ML_Sel(StandardScaler): [ 0  2  4  5  6 10 21 33 35 40]
#F1 RFs:  0.9760
#F1 DTs:  0.9722
#F1 LRs:  0.9651
#F1 Avg:   0.9711

Scaling: StandardScaler
Random Forests
Default threshold:  0.035
The features selected are columns:  [ 0  2  4  5  6 10 21 33 35 40]
F1 RFs:  0.9765
F1 DTs:  0.9722
F1 LRs:  0.9662
F1 Avg:   0.9716


In [9]:
# escolher o conjunto de treino com maior f1_avg
_,X_train, X_test = max(datasets_reduced, key= lambda x : x[0])

## Modelos

Nesta segunda parte iremos criar modelos que consigam prevêr se um químico é ou não biodegradável. Iremos também otimizar estes modelos consoante os seus hiperparâmetros. Os modelos a ser utilizados são: Decision Tree, Regressão Logística, KNN, SVM, Random Forests e XGBoost

In [11]:
# Vamos guardar aqui os modelos e seu respetivo mathews correlaction coef
models = []

**Decision Tree Classifier**

In [12]:
params = [
    {"max_depth" : [6,8,10,12,14,16,18,20,22,24,26,28,30],
    "min_samples_leaf" : [1,2,5,10, 15, 20],
    "min_samples_split" : [2,5,10, 20, 25, 30],
    "criterion":['gini','entropy']}]

grid_search_treeclass = GridSearchCV(
    DecisionTreeClassifier(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_treeclass.fit(X_train,  y_bio_train)
print("Melhores Parâmetros:", grid_search_treeclass.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_treeclass.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_treeclass.best_estimator_))

Melhores Parâmetros: {'criterion': 'gini', 'max_depth': 14, 'min_samples_leaf': 5, 'min_samples_split': 25} 

The Accuracy is:  0.9480
The Precision is:  0.9658
The Recall is:  0.9719
The F1 score is:  0.9689
The Matthews correlation coefficient is:  0.8110

This is the Confusion Matrix
     0     1
0  474    98
1   80  2771


**Regressão Logistica**

In [13]:
params = [
    {"C" : [x*0.1 for x in range(1,11)],
    "max_iter" : [999999]}]

grid_search_log = GridSearchCV(
    LogisticRegression(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_log.fit(X_train, y_bio_train)
print("Melhores Parâmetros:", grid_search_log.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_log.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_log.best_estimator_))

Melhores Parâmetros: {'C': 0.6000000000000001, 'max_iter': 999999} 

The Accuracy is:  0.9366
The Precision is:  0.9419
The Recall is:  0.9846
The F1 score is:  0.9628
The Matthews correlation coefficient is:  0.7581

This is the Confusion Matrix
     0     1
0  399   173
1   44  2807


**Gaussian Naive Bayes**

In [14]:
gaussNB = GaussianNB()
gaussNB.fit(X_train, y_bio_train)
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, gaussNB)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), gaussNB))

The Accuracy is:  0.9179
The Precision is:  0.9547
The Recall is:  0.9463
The F1 score is:  0.9505
The Matthews correlation coefficient is:  0.7104

This is the Confusion Matrix
     0     1
0  444   128
1  153  2698


**KNN**

In [15]:
params = [{"n_neighbors": [1,2,3,4,5,6,7,8,9,10],
          "weights":["uniform", "distance", gaussian]}] #não se usou gaussian pois havia alguns erros

grid_search_knn = GridSearchCV(
    KNeighborsClassifier(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_knn.fit(X_train, y_bio_train)
print("Melhores Parâmetros:", grid_search_knn.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_knn.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_knn.best_estimator_))

Melhores Parâmetros: {'n_neighbors': 4, 'weights': <function gaussian at 0x000001601F1F5A60>} 

The Accuracy is:  0.9568
The Precision is:  0.9668
The Recall is:  0.9818
The F1 score is:  0.9742
The Matthews correlation coefficient is:  0.8407

This is the Confusion Matrix
     0     1
0  476    96
1   52  2799


**SVM**

In [20]:
params =[{"kernel": ['linear','rbf','sigmoid'],
         "gamma": [0.1,0.5,1,10,100,1000],
         "C": [0.1,1,10,100,1000]}]

grid_search_svc = GridSearchCV(
    SVC(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_svc.fit(X_train, y_bio_train)
print("Melhores Parâmetros:", grid_search_svc.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_svc.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_svc.best_estimator_))

Melhores Parâmetros: {'C': 10, 'gamma': 0.5, 'kernel': 'rbf'} 

The Accuracy is:  0.9562
The Precision is:  0.9681
The Recall is:  0.9797
The F1 score is:  0.9738
The Matthews correlation coefficient is:  0.8393

This is the Confusion Matrix
     0     1
0  480    92
1   58  2793


**RANDOM FOREST**

In [17]:
params = [
    {"n_estimators": [10,100,1000],
    "max_depth" : [4,10,16,22,28],
    "min_samples_leaf" : [5,10,20],
    "min_samples_split" : [5,10,20,30],
    "criterion":['gini','entropy']}]

grid_search_rfc = GridSearchCV(
    RandomForestClassifier(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_rfc.fit(X_train,  y_bio_train)
print("Melhores Parâmetros:", grid_search_rfc.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_rfc.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_rfc.best_estimator_))

Melhores Parâmetros: {'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 1000} 

The Accuracy is:  0.9553
The Precision is:  0.9617
The Recall is:  0.9856
The F1 score is:  0.9735
The Matthews correlation coefficient is:  0.8336

This is the Confusion Matrix
     0     1
0  460   112
1   41  2810


**ADABOOST**

In [21]:
params =[{"n_estimators": [10,100,1000],
          "learning_rate": [0.01,0.1,0.5,1],
          "base_estimator": [GaussianNB(), DecisionTreeClassifier()]}]

grid_search_ada = GridSearchCV(
    AdaBoostClassifier(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_ada.fit(X_train, y_bio_train)
print("Melhores Parâmetros:", grid_search_ada.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_ada.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_ada.best_estimator_))

Melhores Parâmetros: {'base_estimator': GaussianNB(), 'learning_rate': 0.1, 'n_estimators': 100} 

The Accuracy is:  0.9422
The Precision is:  0.9480
The Recall is:  0.9846
The F1 score is:  0.9659
The Matthews correlation coefficient is:  0.7810

This is the Confusion Matrix
     0     1
0  418   154
1   44  2807


**XGBOOST**

In [22]:
params=[{"n_estimators": [10,100,500,1000],
        "max_depth" : [4,8,12,16,20,24,28],
        "learning_rate":[0.01,0.1,0.5,1]}]

grid_search_xgb = GridSearchCV(
    XGBClassifier(), params, scoring="f1", cv=kf, n_jobs=-1)

grid_search_xgb.fit(X_train, y_bio_train)
print("Melhores Parâmetros:", grid_search_xgb.best_params_,"\n")
(Truth, Preds) = CrossValidation(X_train, y_bio_train, kf, grid_search_xgb.best_estimator_)
printClassResults(Truth, Preds)
models.append((matthews_corrcoef(Truth, Preds), grid_search_xgb.best_estimator_))

Melhores Parâmetros: {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1000} 

The Accuracy is:  0.9609
The Precision is:  0.9689
The Recall is:  0.9846
The F1 score is:  0.9767
The Matthews correlation coefficient is:  0.8558

This is the Confusion Matrix
     0     1
0  482    90
1   44  2807


## Validação

In [23]:
# Escolhemos automaticamente o melhor modelo baseado no matthews coeficient
best_model = max(models, key= lambda x: x[0])[1]
best_model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [24]:
IVS_Preds = best_model.predict(X_test)
printClassResults(y_bio_test, IVS_Preds)

The Accuracy is:  0.9614
The Precision is:  0.9745
The Recall is:  0.9805
The F1 score is:  0.9775
The Matthews correlation coefficient is:  0.8435

This is the Confusion Matrix
     0    1
0  142   25
1   19  955
